In [1]:
# initializes SQLite
import sqlite3 

conn = sqlite3.connect('housedata.db')
c = conn.cursor()

In [2]:
# Dropping tables from previous run
query = "DROP TABLE IF EXISTS zip_code_database;"
c.execute(query)
conn.commit()
query = "DROP TABLE IF EXISTS kc_house_data;"
c.execute(query)
conn.commit()
query = "DROP TABLE IF EXISTS joined_table;"
c.execute(query)
conn.commit()

In [3]:
# create the two initial tables (if they do not already exist)
query = '''CREATE TABLE IF NOT EXISTS zip_code_database (
    zip TEXT PRIMARY KEY,
    type TEXT,
    decommissioned TEXT,
    primary_city TEXT,
    acceptable_cities TEXT,
    unacceptable_cities TEXT,
    state TEXT,
    county TEXT,
    timezone TEXT,
    area_codes TEXT,
    world_region TEXT,
    country TEXT,
    latitude TEXT,
    longitude TEXT,
    irs_estimated_population_2015 INTEGER
    );'''

c.execute(query)
conn.commit()

query = '''CREATE TABLE IF NOT EXISTS kc_house_data (
    id TEXT PRIMARY KEY,
    date DATE,
    price INTEGER,
    bedrooms INTEGER,
    bathrooms TEXT,
    sqft_living TEXT,
    sqft_lot INTEGER,
    floors INTEGER,
    waterfront TEXT,
    view TEXT,
    condition INTEGER,
    grade INTEGER,
    sqft_above INTEGER,
    sqft_basement INTEGER,
    yr_built TEXT,
    yr_renovated TEXT,
    zipcode TEXT,
    lat TEXT,
    long TEXT,
    sqft_living15 INTEGER,
    sqft_lot15 INTEGER
    );'''

c.execute(query)
conn.commit()

In [4]:
# populates zipcode_database table
import xlrd
file_name = "zip_code_database.xls"
print (f"Reading {file_name}")
fp = xlrd.open_workbook(file_name)
sh = fp.sheet_by_index(0)
print(f"The excel file has {sh.nrows} rows.")  
data_list = []

for row_number in range(sh.nrows):

    line = sh.row_values(row_number)


    if row_number == 0: # on the header line
        field_name_list = ",".join(line)
    else: # on a data line

        # special formatting for handling the zip code transfer between systems. XLRD is reading the zip as a number and converting it to a float. Force it to be a string 
        zip_code = str(int(line[0]))
        # prepend with zeros
        zip_code = ('0'*(5-len(zip_code))) + zip_code
        line[0] = zip_code

        data_list.append(line)

print(f"\nFilling zip_code_database with {len(data_list)} records.")
# write the data to our db
query = f"INSERT INTO zip_code_database ({field_name_list}) VALUES({'?,'*(len(data_list[0])-1)}?);"
c.executemany(query,data_list)
conn.commit()
print("Finished filling zip_code_database")

Reading zip_code_database.xls
The excel file has 42633 rows.

Filling zip_code_database with 42632 records.
Finished filling zip_code_database


In [5]:
# populate KC_house_data table and removes duplicates
print ("Populating kc_house_data table....")
with open ("kc_house_data.csv","r") as f:

    count = 0 # line counter
    data_list = []

    # encountered a problem with duplicate ids upon attempt to commit to DB. Making a list of ids and checking for duplicates during import.
    id_list = []
    duplicates = []

    while True: 
        count += 1
    
        # Get next line from file 
        line = f.readline().strip()
    
        # if line is empty end of file is reached 
        if not line: 
            break

        if count == 1: # on the header line
            field_name_list = line
        else: # on a data line
            data = tuple(line.split(','))
            if (data[0] in id_list):
                #print('X',end="",flush=True) #shows when a duplicate id has been found
                duplicates.append(data[0])
                continue
            id_list.append(data[0])
            data_list.append(data)
            #print('.',end="",flush=True)

# write the data to our db
query = f"INSERT INTO kc_house_data ({field_name_list}) VALUES({'?,'*(len(data_list[0])-1)}?);"
c.executemany(query,data_list)
conn.commit()

print ("\n... finished.")

Populating kc_house_data table....

... finished.


In [6]:
print (f"Omitted {len(duplicates)} duplicates: \n", '\n'.join(duplicates))

Omitted 177 duplicates: 
 6021501535
4139480200
7520000520
3969300030
2231500030
8820903380
726049190
8682262400
9834200885
8062900070
1231000510
6117501820
2228900270
3630120700
7888000390
2143700830
3395040550
4345000510
9407110710
1000102
1781500435
1232000810
7856400240
7983000200
9136103130
5083000375
7409700215
1823049202
2767602141
1254200015
723049156
4435000705
4364700600
1825069031
3262300940
9353300600
1524079093
9809000020
3303000130
7524400250
4204400098
6632900574
2726049071
3578401060
1788800630
6381500170
1954420170
1545800290
9828200460
9222400605
8832900780
8645530010
3323059027
5417600130
8651510380
5249801440
526059224
1446403850
251300110
4139420590
2621600015
2206700215
2787460720
3883800011
4154300296
1721801010
1237500540
5101405604
5127001320
5282200015
7977201065
3739500096
1568100300
1922059278
302000375
722039087
8129700644
4443800385
2212200100
4302201085
5054800110
8103000110
2568300045
3598600049
1219000473
3293700496
8945100320
2422049104
1995200200
1450

In [7]:
# Create inner join table from the above tables

query=''' 
    CREATE TABLE joined_table AS
    SELECT * FROM zip_code_database 
    INNER JOIN kc_house_data
    ON kc_house_data.zipcode = zip_code_database.zip;
    '''
c.execute(query)

conn.close()